## Import Python libraries

In [ ]:
!pip install flwr

In [2]:
import tensorflow.compat.v1 as tf
sess = tf.Session()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input"))
import errno
import tensorflow as tf
import csv
from keras import backend as K
import sklearn
from tensorflow import keras
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPool1D
from keras.optimizers import Adam
import random
import pandas as pd
import numpy as np
import math
import os
import flwr as fl
import array
from scipy.stats import dirichlet
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['adult-census-income']


ModuleNotFoundError: No module named 'flwr'

In [ ]:
!pip install aif360 

In [ ]:
!pip install BlackBoxAuditing

## Import dataset

In [4]:
path = ('/kaggle/input/adult-census-income/adult.csv')
df = pd.read_csv(path, encoding='latin-1')

In [5]:
df.race.unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo'], dtype=object)

## Exploratory Data Analysis

### Check shape of dataset

In [ ]:
df.shape

We can see that there are 32561 instances and 15 attributes in the data set.

### Preview dataset

In [ ]:
df.head()

In [ ]:
df.race.value_counts()

### View summary of dataframe

In [ ]:
df.info()

Summary of the dataset shows that there are no missing values. But the preview shows that the dataset contains values coded as `?`. So, I will encode `?` as NaN values.

### Encode `?` as `NaNs`

In [ ]:
df[df == '?'] = np.nan

In [ ]:
df.head()

In [ ]:
df.isna().sum()

### Impute missing values with mode

In [ ]:
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)

### Check again for missing values

In [ ]:
df.isnull().sum()

In [6]:
import numpy as np
import pandas as pd

def race_Dirichelet_sampling(data, alphas, values, n_lines):
    assert (n_lines <= len(data))
    assert (len(values) == len(alphas))

    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    groups = []

    for i in range(len(values)):
        if round(n_lines * s[i]) > 0:
            groups.append(data[data['race'] == values[i]].sample(n=round(n_lines * s[i]), replace=True))
        else:
            groups.append(data[data['race'] == values[i]].sample(n=1))

    return pd.concat(groups)
alphas = [1, 2, 3, 1, 1]  
values = ['White', 'Black', 'Asian-Pac-Islander', 'Other', 'Amer-Indian-Eskimo']
n_lines = 1000 
result_dataset = race_Dirichelet_sampling(df, alphas, values, n_lines)


In [8]:
alphas = [1, 2, 3, 1, 1]  
values = ['White', 'Black', 'Asian-Pac-Islander', 'Other', 'Amer-Indian-Eskimo']
n_lines = 1000 
result_dataset = race_Dirichelet_sampling(df, alphas, values, n_lines)


In [13]:
df.race.value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

In [12]:
df.race.value_counts()
result_dataset.race.value_counts()

Asian-Pac-Islander    459
Black                 403
Amer-Indian-Eskimo     90
Other                  47
White                   1
Name: race, dtype: int64

In [ ]:
def Dirichelet_sampling(data, alphas, values, n_lines):
    """
        Produit une dataset suivant la distribution de Dirichlet paramétrée par le vecteur alphas
       sur les valeurs values
        e.g. values = [Male, Female] --> Le nombre d'individus correspondant à Male et Female va suivre la
        distrib Dir(alphas)
    """
    # values = data[col].unique()
    assert (n_lines <= len(data.axes[0]))
    assert (len(values) == len(alphas))
    print(len(data.axes[0]))
    # sample a distribution from dir(alphas)
    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    print(s)
    print(alphas)
    groups = []
    for i in range(len(values)):
        if values[0] == 'Male' or values[0] == 'Female':
            if round(n_lines * s[0]) > 0:
                groups.append(data[data['sex'] == 1.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

            if round(n_lines * s[1]) > 0:
                groups.append(data[data['sex'] == 0.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

        else:
            if round(n_lines * s[i]) > 0:
                groups.append(data[data[values[i]] == 1.0].sample(n=round(n_lines * s[i]), replace=True))
            else:
                groups.append(data[data[values[i]] == 1.0].sample(n=1))

    return pd.concat(groups)

In [ ]:
df.race.unique()

Now we can see that there are no missing values in the dataset.

### Setting feature vector and target variable

In [ ]:
X = df.drop(['income'], axis=1)
y = df['income']

In [ ]:
y.unique(),y.nunique()

In [ ]:
X.head()

## Split data into separate training and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Feature Engineering

### Encode categorical variables: LabelEncoder

![](https://ai-ml-analytics.com/wp-content/uploads/2020/08/encoding-3.png)

In [ ]:
from sklearn import preprocessing

categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [ ]:
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)
dff = pd.concat([X, y], axis=1)
dff=dff.reset_index(drop=True)

In [ ]:
df.race.unique()

In [ ]:
d={"White":1,"Black":0,"Asian-Pac-Islander":0,"Other":0,'Amer-Indian-Eskimo':0}

In [ ]:
for i in range(5,df.shape[0]):
    print(i)
    print(df.loc[i,"race"])
    df.loc[i,"race"] = d[df.loc[i,"race"]]

In [ ]:
def Dirichelet_sampling(data, alphas, values, n_lines):
    """
        Produit une dataset suivant la distribution de Dirichlet paramétrée par le vecteur alphas
       sur les valeurs values
        e.g. values = [Male, Female] --> Le nombre d'individus correspondant à Male et Female va suivre la
        distrib Dir(alphas)
    """
    # values = data[col].unique()
    assert (n_lines <= len(data.axes[0]))
    assert (len(values) == len(alphas))
    print(len(data.axes[0]))
    # sample a distribution from dir(alphas)
    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    print(s)
    print(alphas)
    groups = []
    for i in range(len(values)):
        if values[0] == 'white' :
            if round(n_lines * s[0]) > 0:
                groups.append(data[data['race'] == 1.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['race'] == 1.0].sample(n=1))

            if round(n_lines * s[1]) > 0:
                groups.append(data[data['race'] == 0.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['race'] == 1.0].sample(n=1))

        else:
            if round(n_lines * s[i]) > 0:
                groups.append(data[data[values[i]] == 1.0].sample(n=round(n_lines * s[i]), replace=True))
            else:
                groups.append(data[data[values[i]] == 1.0].sample(n=1))

    return pd.concat(groups)

In [ ]:
def Dirichelet_sampling(data, alphas, values, n_lines):
    """
        Produit une dataset suivant la distribution de Dirichlet paramétrée par le vecteur alphas
       sur les valeurs values
        e.g. values = [Male, Female] --> Le nombre d'individus correspondant à Male et Female va suivre la
        distrib Dir(alphas)
    """
    # values = data[col].unique()
    assert (n_lines <= len(data.axes[0]))
    assert (len(values) == len(alphas))
    print(len(data.axes[0]))
    # sample a distribution from dir(alphas)
    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    print(s)
    print(alphas)
    groups = []
    for i in range(len(values)):
        if values[0] == 'Male' or values[0] == 'Female':
            if round(n_lines * s[0]) > 0:
                groups.append(data[data['sex'] == 1.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

            if round(n_lines * s[1]) > 0:
                groups.append(data[data['sex'] == 0.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

        else:
            if round(n_lines * s[i]) > 0:
                groups.append(data[data[values[i]] == 1.0].sample(n=round(n_lines * s[i]), replace=True))
            else:
                groups.append(data[data[values[i]] == 1.0].sample(n=1))

    return pd.concat(groups)

In [ ]:
values = ['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo']
df1 = Dirichelet_sampling(dff, [0.2,0.8], values, 11000)
df2 = Dirichelet_sampling(dff, [0.7,0.9], values, 10000)
df3 = Dirichelet_sampling(dff, [0.6,0.3], values, 10000)

In [ ]:
df.race.unique()

In [ ]:
df1.shape,df2.shape, df3.shape

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

## Feature Scaling

![](https://cdn-images-1.medium.com/max/370/1*Nlgc_wq2b-VfdawWX9MLWA.png)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [ ]:
X_train.head()

## Modeling + Fairness

**Remove the 'race' and 'sex' columns from the dataset to remove any potential biases or discriminatory factors**

In [ ]:
X_trainn = X_train.drop(['race', 'sex'], axis=1)
X_testt = X_test.drop(['race', 'sex'], axis=1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
model = DecisionTreeRegressor()
model.fit(X_trainn, y_train)

y_pred = model.predict(X_testt)

mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

****

# adversarial debiasing :
is an in-processing technique for fairness in machine learning that tries to learn a fair classifier by adding an adversary to the training process.

The goal is to train a model that is both accurate and fair with respect to some protected attributes, such as race or gender. Adversarial debiasing adds a new step to the training process, where an adversarial network is trained to predict the protected attribute from the hidden layers of the main network. This adversarial network tries to maximize the error in predicting the protected attribute while the main network tries to minimize it.

By doing so, the main network learns to encode the input features in a way that is less informative about the protected attributes, which leads to a fairer classifier.

In [ ]:
!pip install aif360

from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

y_train_df = pd.DataFrame({'income': y_train})

dataset_train = StandardDataset(
    df=X_train.join(y_train_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']]
)

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
debias_model = AdversarialDebiasing(
    unprivileged_groups=[{'race': 0, 'sex': 0}],
    privileged_groups=[{'race': 1, 'sex': 1}],
    scope_name='adversarial_debiasing',
    sess=sess
)
debias_model.fit(dataset_train)

In [ ]:
y_test_df = pd.DataFrame({'income': y_test})
dataset_test = StandardDataset(
    df=X_test.join(y_test_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']]
)
dataset_debiased = debias_model.predict(dataset_test)

In [ ]:
# Evaluate the performance of the debiased model
y_pred = dataset_debiased.labels
y_true = dataset_test.labels.ravel()
print("Accuracy score:", accuracy_score(y_true, y_pred))
print("Balanced accuracy score:", balanced_accuracy_score(y_true, y_pred))
print("F1 score:", f1_score(y_true, y_pred))

Accuracy score: measures the proportion of correctly classified instances among all instances. It is computed as the number of true positives plus true negatives divided by the total number of instances. It is not suitable for imbalanced datasets, where the classes have very different sizes.

Balanced accuracy score: a modified version of accuracy score that takes into account the class imbalance by computing the average of the sensitivity (true positive rate) and specificity (true negative rate). It is computed as (sensitivity + specificity) / 2.

F1 score: measures the trade-off between precision and recall. It is the harmonic mean of precision and recall, computed as 2 * (precision * recall) / (precision + recall). It is a good metric for imbalanced datasets, where the focus is on correctly identifying the minority class.

****

# Disparate Impact Remover
is a pre-processing algorithm used to mitigate the effects of sensitive attributes (such as race or gender) on the outcomes of machine learning models. The algorithm works by adjusting the distribution of the sensitive attribute in the dataset to achieve a desired level of fairness.

The Disparate Impact Remover algorithm works by creating a new dataset in which the sensitive attribute (such as race or gender) is balanced across the different values of the outcome variable. This is achieved by adjusting the probability of selecting each example in the training dataset, such that the overall proportion of the sensitive attribute is the same across all possible outcomes. This approach is based on the intuition that if the dataset is balanced with respect to the sensitive attribute, then the machine learning model will not be able to discriminate based on this attribute.

In [ ]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
train_dataset = BinaryLabelDataset(df=X_train.join(y_train_df), label_names=['income'], 
                                   protected_attribute_names=['race', 'sex'], 
                                   favorable_label=1, unfavorable_label=0)
test_dataset = BinaryLabelDataset(df=X_test.join(y_test_df), label_names=['income'], 
                                  protected_attribute_names=['race', 'sex'], 
                                  favorable_label=1, unfavorable_label=0)

In [ ]:
DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute='race')
train_repaired_dataset = DIR.fit_transform(train_dataset)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_repaired_dataset.features, train_repaired_dataset.labels.ravel())

In [ ]:
y_pred = rfc.predict(test_dataset.features)

In [ ]:
accuracy = accuracy_score(test_dataset.labels.ravel(), y_pred)
precision = precision_score(test_dataset.labels.ravel(), y_pred)
recall = recall_score(test_dataset.labels.ravel(), y_pred)
f1 = f1_score(test_dataset.labels.ravel(), y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

****

# Learning fair representations (LFR) 
is a technique for learning representations of data that are optimized to remove or reduce the impact of sensitive attributes such as race or gender. It is a type of pre-processing technique that can be applied to the data before it is used to train a machine learning model.

The goal of LFR is to find a representation of the data that separates the information that is relevant to the prediction task from the information that is associated with the sensitive attribute. This representation can then be used as input to a machine learning model that can make predictions without relying on the sensitive attribute.

In [ ]:
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
train_dataset  = StandardDataset(
    df=X_train.join(y_train_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']])
test_dataset  = StandardDataset(
    df=X_test.join(y_test_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = rfc.predict(test_dataset.features)
accuracy = accuracy_score(test_dataset.labels.ravel(), y_pred)
precision = precision_score(test_dataset.labels.ravel(), y_pred)
recall = recall_score(test_dataset.labels.ravel(), y_pred)
f1 = f1_score(test_dataset.labels.ravel(), y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


Reweighing

In [ ]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

# Création des jeux de données d'entraînement et de test avec AIF360
train_dataset = BinaryLabelDataset(df=X_train.join(y_train_df), label_names=['income'], 
                                   protected_attribute_names=['race', 'sex'], 
                                   favorable_label=1, unfavorable_label=0)

test_dataset = BinaryLabelDataset(df=X_test.join(y_test_df), label_names=['income'], 
                                  protected_attribute_names=['race', 'sex'], 
                                  favorable_label=1, unfavorable_label=0)

# Définition des groupes privilégiés et non privilégiés
privileged_groups = [{'race': 1}]  # Remplacez par vos propres valeurs
unprivileged_groups = [{'race': 0}]  # Remplacez par vos propres valeurs

# Application de la méthode de reweighing
rw = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
train_dataset_reweighed = rw.fit_transform(train_dataset)

# Entraînement d'un modèle de RandomForestClassifier sur les données reweighed
rfc = RandomForestClassifier()
rfc.fit(train_dataset_reweighed.features, train_dataset_reweighed.labels.ravel())

# Prédictions sur l'ensemble de test
X_test_reweighed = rw.transform(test_dataset)
y_pred = rfc.predict(X_test_reweighed.features)

# Calcul des mesures de performance
accuracy = accuracy_score(test_dataset.labels.ravel(), y_pred)
precision = precision_score(test_dataset.labels.ravel(), y_pred)
recall = recall_score(test_dataset.labels.ravel(), y_pred)
f1 = f1_score(test_dataset.labels.ravel(), y_pred)

print("Exactitude:", accuracy)
print("Précision:", precision)
print("Rappel:", recall)
print("Score F1:", f1)


In [ ]:
train_dataset_reweighed

In [ ]:
def re(X_train,y_train_df ,X_test,y_test_df):
    train_dataset = BinaryLabelDataset(df=X_train.join(y_train_df), label_names=['income'], 
                                   protected_attribute_names=['race', 'sex'], 
                                   favorable_label=1, unfavorable_label=0)

    test_dataset = BinaryLabelDataset(df=X_test.join(y_test_df), label_names=['income'], 
                                  protected_attribute_names=['race', 'sex'], 
                                  favorable_label=1, unfavorable_label=0)
    privileged_groups = [{'race': 1}]  # Remplacez par vos propres valeurs
    unprivileged_groups = [{'race': 0}]  # Remplacez par vos propres valeurs

    # Application de la méthode de reweighing
    rw = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    train_dataset_reweighed = rw.fit_transform(train_dataset)
    X_test_reweighed = rw.transform(test_dataset)
    return train_dataset_reweighed X_test_reweighed
